<a href="https://colab.research.google.com/github/Goldeno10/flexisaf_Internship_GenAI_DS_Intermediate/blob/main/task_3/tast_3_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("http://kaggle.com/datasets/thoughtvector/customer-support-on-twitter")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: goldenoelgenius
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter


100%|██████████| 169M/169M [00:00<00:00, 972MB/s] 

In [ ]:
import re
import nltk
import string
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

full_df = pd.read_csv(
    "customer-support-on-twitter/twcs/twcs.csv",
    nrows=5000
  )

df = full_df[["text"]]
df["text"] = df["text"].astype(str)

# full_df.head()

df.head()

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


# Lower Casing

In [ ]:
df["text_lower"] = df["text"].str.lower()
df.tail()

,text,text_lower
4995,"@117290 Hm, they should definitely resume if a...","@117290 hm, they should definitely resume if a..."
4996,"@117290 That's where they need to be on, so ch...","@117290 that's where they need to be on, so ch..."
4997,@115940 Can you guys maybe like....FIX your st...,@115940 can you guys maybe like....fix your st...
4998,@117291 Hulu is only available in the U.S. rig...,@117291 hulu is only available in the u.s. rig...
4999,@115940 when will the platform available in la...,@115940 when will the platform available in la...


# Removal of Punctuations

In [ ]:
df.drop(["text_lower"], axis=1,  inplace=True)

PUNCTUATION_TO_REMOVE = string.punctuation

def remove_punctuation(text: str):
  """Removes puntuations text 'text'"""
  return text.translate(
      str.maketrans("", "", PUNCTUATION_TO_REMOVE)
    )

df["text_wo_punct"] = df["text"].apply(
    lambda text: remove_punctuation(text)
  )

df.head()

,text,text_wo_punct
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...
4,@sprintcare I did.,sprintcare I did


# Removal of Stop Words

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

", ".join(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"a, about, above, after, again, against, ain, all, am, an, and, any, are, aren, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can, couldn, couldn't, d, did, didn, didn't, do, does, doesn, doesn't, doing, don, don't, down, during, each, few, for, from, further, had, hadn, hadn't, has, hasn, hasn't, have, haven, haven't, having, he, he'd, he'll, her, here, hers, herself, he's, him, himself, his, how, i, i'd, if, i'll, i'm, in, into, is, isn, isn't, it, it'd, it'll, it's, its, itself, i've, just, ll, m, ma, me, mightn, mightn't, more, most, mustn, mustn't, my, myself, needn, needn't, no, nor, not, now, o, of, off, on, once, only, or, other, our, ours, ourselves, out, over, own, re, s, same, shan, shan't, she, she'd, she'll, she's, should, shouldn, shouldn't, should've, so, some, such, t, than, that, that'll, the, their, theirs, them, themselves, then, there, these, they, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, 

In [ ]:
STOP_WORDS = set(stopwords.words("english"))

def remove_stopwords(text):
  """Remove stopwords in text 'text'"""
  return " ".join(
      [word for word in str(text).split() if word not in STOP_WORDS]
    )

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))

df.head()


,text,text_wo_punct,text_wo_stop
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...
4,@sprintcare I did.,sprintcare I did,sprintcare I


# Removal of Frequent Words

In [ ]:
from collections import Counter

cnt = Counter()

for text in df["text_wo_stop"].values:
  for word in text.split():
    cnt[word] += 1

cnt.most_common(10)

[('I', 1437),
 ('us', 752),
 ('DM', 514),
 ('help', 479),
 ('Please', 376),
 ('We', 338),
 ('Hi', 293),
 ('Thanks', 287),
 ('get', 279),
 ('please', 247)]

In [ ]:
FREQ_WORDS = set([w for (w,wc) in cnt.most_common(10)])

def remove_freq_words(text):
  return " ".join(
      [word for word in str(text).split() if word not in FREQ_WORDS]
    )

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(
    lambda text: remove_freq_words(text)
  )

df.head()

,text,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare I did,sprintcare I,sprintcare


# Removal of Rare Words

In [ ]:
df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RARE_WORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])

def remove_rare_words(text):
  """Remove rare words in a text 'text'"""
  return " ".join(
      [word for word in str(text).split() if word not in RARE_WORDS]
    )

df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(
    lambda text: remove_rare_words(text)
)

df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare
0,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 send Private Message assist Just click ...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare,sprintcare


# Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer

df.drop(["text_wo_stopfreq", "text_wo_stopfreqrare"], axis=1, inplace=True)

stemmer = PorterStemmer()

def stem_words(text):
  return " ".join(
      [stemmer.stem(word) for word in text.split()]
    )

df["text_stemmed"] = df["text"].apply(
    lambda text: stem_words(text)
  )

df.tail()

,text,text_stemmed
4995,"@117290 Hm, they should definitely resume if a...","@117290 hm, they should definit resum if alrea..."
4996,"@117290 That's where they need to be on, so ch...","@117290 that' where they need to be on, so che..."
4997,@115940 Can you guys maybe like....FIX your st...,@115940 can you guy mayb like....fix your stuf...
4998,@117291 Hulu is only available in the U.S. rig...,@117291 hulu is onli avail in the u.s. right n...
4999,@115940 when will the platform available in la...,@115940 when will the platform avail in latina...


We can see that words like private and propose have their e at the end chopped off due to stemming. This is not intented. What can we do fort hat? We can use Lemmatization in such cases.

Also this porter stemmer is for English language. If we are working with other languages, we can use snowball stemmer. The supported languages for snowball stemmer are



In [ ]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

# Lemmatization

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_words(text: str):
  return " ".join(
      [lemmatizer.lemmatize(word) for word in text.split()]
    )

df["text_lemmatized"] = df["text"].apply(
    lambda text: lemmatize_words(text)
  )

df.tail()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,text_stemmed,text_lemmatized
4995,"@117290 Hm, they should definitely resume if a...","@117290 hm, they should definit resum if alrea...","@117290 Hm, they should definitely resume if a..."
4996,"@117290 That's where they need to be on, so ch...","@117290 that' where they need to be on, so che...","@117290 That's where they need to be on, so ch..."
4997,@115940 Can you guys maybe like....FIX your st...,@115940 can you guy mayb like....fix your stuf...,@115940 Can you guy maybe like....FIX your stu...
4998,@117291 Hulu is only available in the U.S. rig...,@117291 hulu is onli avail in the u.s. right n...,@117291 Hulu is only available in the U.S. rig...
4999,@115940 when will the platform available in la...,@115940 when will the platform avail in latina...,@115940 when will the platform available in la...


In [ ]:
lemmatizer.lemmatize("running", "v")

'run'

In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('averaged_perceptron_tagger_eng')

lemmatizer = WordNetLemmatizer()
wordnet_map = {
    "N": wordnet.NOUN, "V": wordnet.VERB,
    "J": wordnet.ADJ, "R": wordnet.ADV
  }

def lemmatize_words(text: str):
  pos_tagged_text = nltk.pos_tag(text.split())
  return " ".join(
      [
        lemmatizer.lemmatize(
          word, wordnet_map.get(pos[0], wordnet.NOUN))
        for word, pos in pos_tagged_text
      ]
  )

df["text_lemmatized"] = df["text"].apply(
    lambda text: lemmatize_words(text)
  )

df.tail()

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


,text,text_stemmed,text_lemmatized
4995,"@117290 Hm, they should definitely resume if a...","@117290 hm, they should definit resum if alrea...","@117290 Hm, they should definitely resume if a..."
4996,"@117290 That's where they need to be on, so ch...","@117290 that' where they need to be on, so che...","@117290 That's where they need to be on, so ch..."
4997,@115940 Can you guys maybe like....FIX your st...,@115940 can you guy mayb like....fix your stuf...,@115940 Can you guy maybe like....FIX your stu...
4998,@117291 Hulu is only available in the U.S. rig...,@117291 hulu is onli avail in the u.s. right n...,@117291 Hulu be only available in the U.S. rig...
4999,@115940 when will the platform available in la...,@115940 when will the platform avail in latina...,@115940 when will the platform available in la...


# Removal of Emojis

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_emoji("game is on 🔥🔥")

'game is on '

# Removal of Emoticons

There is a minor difference between emojis and emoticons.

From Grammarist.com, emoticon is built from keyboard characters that when put together in a certain way represent a facial expression, an emoji is an actual image.

:-) is an emoticon

😀 is an emoji

In [ ]:
import re
from nltk.tokenize.casual import EMOTICONS

def remove_emoticons(text):
    # EMOTICONS is already a regex string, compile it directly with re.VERBOSE for readability
    emoticon_pattern = re.compile(EMOTICONS, flags=re.VERBOSE)
    return emoticon_pattern.sub(r'', text)

remove_emoticons("Hello :-)")

'Hello '

In [ ]:
remove_emoticons("I am sad :(")

'I am sad '

# Conversion of Emoticon to Words

In the previous step, we have removed the emoticons. In case of use cases like sentiment analysis, the emoticons give some valuable information and so removing them might not be a good solution. What can we do in such cases?

One way is to convert the emoticons to word format so that they can be used in downstream modeling processes. Thanks for Neel again for the wonderful dictionary that we have used in the previous step. We are going to use that again for conversion of emoticons to words.

In [ ]:
import re
try:
  from emot.emo_unicode import EMOTICONS_EMO
except ModuleNotFoundError:
  !pip install emot
  from emot.emo_unicode import EMOTICONS_EMO


def convert_emoticons(text):
    # 1. Sort by length (longest first) to prevent ":)" from matching inside ":-)"
    sorted_emotes = sorted(EMOTICONS_EMO.items(), key=lambda x: len(x[0]), reverse=True)

    for emot_char, emot_word_desc in sorted_emotes:
        # 2. Escape special characters for regex
        escaped_emot_char = re.escape(emot_char)

        # 3. Format description: "Happy face" -> "happy_face"
        formatted_word = "_".join(emot_word_desc.replace(",", "").lower().split())

        # 4. Use Lookarounds (?<!\w) and (?!\w)
        # This ensures we don't match emoticons inside words (like 'fine:D' stays)
        # but matches them next to punctuation like 'Hello!:)'
        pattern = re.compile(rf'(?<![a-zA-Z0-9]){escaped_emot_char}(?![a-zA-Z0-9])')
        text = pattern.sub(formatted_word, text)

    return text


text = "Hello! :-) :-) This works :D"
print(convert_emoticons(text))

Hello! happy_face_smiley happy_face_smiley This works laughing_big_grin_or_laugh_with_glasses


# Conversion of Emoji to Words¶


In [ ]:
import re
try:
  from emot.emo_unicode import UNICODE_EMOJI
except ModuleNotFoundError:
  !pip install emot
  from emot.emo_unicode import UNICODE_EMOJI

def convert_emojis(text):
    # Sort emojis by length descending to prevent partial matches
    # e.g., if one emoji is a substring of another (less common but good practice)
    sorted_emojis = sorted(UNICODE_EMOJI.keys(), key=len, reverse=True)

    for emot_char in sorted_emojis:
        # Escape the emoji character to treat it as a literal in regex
        escaped_emot_char = re.escape(emot_char)

        # Get the textual description, clean it, and format it
        emot_word_desc = UNICODE_EMOJI[emot_char]
        formatted_word = "_".join(emot_word_desc.replace(",", "").replace(":", "").split())

        # Replace the emoji with its formatted word
        text = re.sub(r'(' + escaped_emot_char + r')', formatted_word, text)
    return text

text = "game is on 🔥"
print(convert_emojis(text))

game is on fire


In [ ]:
text = "Hilarious 😂"
convert_emojis(text)